# 2. 讀取netCDF資料

## 讀取單一netCDF檔案

讀取檔案的指令為 **`xarray.open_dataset(filename)`**。

**Example 1:** 讀取OLR檔案。
在正式讀進Python之前，我們先利用`ncdump`指令來了解一下這個檔案的樣貌。

In [1]:
import os 
os.system('ncdump -h data/olr.nc') 

netcdf olr {
dimensions:
	time = UNLIMITED ; // (8760 currently)
	lon = 360 ;
	bnds = 2 ;
	lat = 90 ;
variables:
	double time(time) ;
		time:standard_name = "time" ;
		time:long_name = "reference time" ;
		time:units = "days since 1979-1-1 00:00:00" ;
		time:calendar = "standard" ;
		time:axis = "T" ;
	float lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:bounds = "lon_bnds" ;
	float lon_bnds(lon, bnds) ;
	float lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:bounds = "lat_bnds" ;
	float lat_bnds(lat, bnds) ;
	float olr(time, lat, lon) ;
		olr:standard_name = "toa_outgoing_longwave_flux" ;
		olr:long_name = "NOAA Climate Data Record of Daily Mean Upward Longwave Flux at Top of the Atmosphere" ;
		olr:units = "W m-2" ;
		olr:_FillValue = 0.f ;
		olr:missing_value = 0.f ;
		olr:cell_methods = "time: mean area: mean" ;

0

一個netCDF檔案中，會儲存資料本身，以及它的座標軸。時間軸`time`是double的格式，如果將時間軸的內容顯示出來，我們會看到

`os.system('ncdump -v time data/olr.nc')`

~~~
double time(time) ;
	time:standard_name = "time" ;
	time:long_name = "reference time" ;
	time:units = "days since 1979-1-1 00:00:00" ;
	time:calendar = "standard" ;
	time:axis = "T" ;
......
time = 6940, 6941, 6942, 6943, 6944, 6945, 6946, 6947, 6948, 6949, 6950, 
    6951, 6952, 6953, 6954, 6955, 6956, 6957, 6958, 6959, 6960, 6961, 6962, 
~~~ 

列印出的時間是15305, 15306, 15307, ...這樣的時間序列，再看時間單位`time:units = "days since 1979-1-1 00:00:00"`，這表示時間格式是相對於1979年1月1日0時是第幾天，我們把這樣儲存時間的方法稱為**相對時間**。

了解了netCDF儲存檔案的方式後，我們用xarray來開啟檔案看看。xarray讀取netCDF檔案，是利用`xr.open_dataset`的指令，也就是把檔案讀進Dataset的資料格式中，

In [2]:
import xarray as xr

olr_ds = xr.open_dataset("data/olr.nc")
olr_ds

<xarray.Dataset>
Dimensions:   (time: 8760, lon: 360, bnds: 2, lat: 90)
Coordinates:
  * time      (time) datetime64[ns] 1998-01-01 1998-01-02 ... 2021-12-31
  * lon       (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat       (lat) float32 -44.5 -43.5 -42.5 -41.5 ... 41.5 42.5 43.5 44.5
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (lon, bnds) float32 ...
    lat_bnds  (lat, bnds) float32 ...
    olr       (time, lat, lon) float32 ...
Attributes: (12/49)
    CDI:                        Climate Data Interface version 1.9.10 (https:...
    Conventions:                CF-1.6
    source:                     NOAA Archive of HIRS L1B data from TIROS-N Se...
    institution:                UMD/ESSIC > Earth System Science Interdiscipl...
    history:                    Fri Jan 14 11:02:54 2022: cdo sellonlatbox,0,...
    conventions:                CF-1.6
    ...                         ...
    Metadata_Link:              gov.noaa.ncdc:C00875
    product_version:            Ver01Rev02
    platform:                   TIROS-N > Television Infrared Observation Sat...
    sensor:                     HIRS-2 > High Resolution Infra-red Sounder/2,...
    spatial_resolution:         1.0 by 1.0 degree equal angle
    CDO:                        Climate Data Operators version 1.9.10 (https:...

我們會得到一個xarray.Dataset，裡面存放olr這個**變數 (variable)**，以及olr陣列的**座標軸 (coordinates)**，座標軸的名稱稱為**維度 (dimension)**。

我們發現，`time`時間座標軸的內容，已經被轉化成我們容易辨識的時間格式yyyy-mm-dd了，這是因為xarray會自動轉譯netCDF檔中的時間，變成**datetime object**，這樣的時間格式稱為**絕對時間**。如此一來，之後要選取時間，就可以直接用年月日來處理，不必自行轉換相對時間和絕對時間。

## 讀取多個netCDF檔案 (Reading Multiple Files)

有時候我們要處理長時間的氣候資料，但作業中心提供的資料卻是一年一個檔案，如果為每一筆資料都開一個Dataset，操作起來不方便，因此xarray可以用 `xarray.open_mfdataset(paths)`指令來一次開啟所有檔案，並且選擇`combine = 'nested'`，讓檔案沿著指定的坐標軸合併。

**Example 2 沿座標軸合併：** 開啟NCEP R2風場資料，其中檔案是一年一筆。

同樣地，我們先打開一筆資料看一下。

In [3]:
os.system('ncdump -h data/ncep_r2_uv850/u850.1998.nc')

netcdf u850.1998 {
dimensions:
	time = UNLIMITED ; // (365 currently)
	bnds = 2 ;
	lon = 144 ;
	lat = 73 ;
	level = 1 ;
variables:
	double time(time) ;
		time:standard_name = "time" ;
		time:long_name = "Time" ;
		time:bounds = "time_bnds" ;
		time:units = "hours since 1800-1-1 00:00:0.0" ;
		time:calendar = "standard" ;
		time:axis = "T" ;
	double time_bnds(time, bnds) ;
	float lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "Longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
	float lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "Latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
	float level(level) ;
		level:standard_name = "air_pressure" ;
		level:long_name = "Level" ;
		level:units = "millibar" ;
		level:positive = "down" ;
		level:axis = "Z" ;
		level:actual_range = 1000.f, 10.f ;
		level:GRIB_id = 100s ;
		level:GRIB_name = "hPa" ;
		level:coordinate_defines = "point" ;
	short uwnd(time, level, lat, lon) ;
		uwnd:standard_

0

不同於剛剛OLR的檔案，變數`olr`是float格式，這邊提供的風場資料`uwnd`是short格式。short是一種壓縮的格式，要透過`add_offset`和`scale_factor`轉換變成原始的數值，這兩個值都儲存在uwnd的attribute中，轉換的公式為

pck = (upk - add_offset) / scale_factor

接下來我們直接打開檔案看看。

In [4]:
uds = (xr.open_mfdataset( 'data/ncep_r2_uv850/u850.*.nc',    # 檔案名稱
                           combine = "by_coords",               
                           parallel=True                     # 運用dask平行運算，提高運算效率
                         ))       
uds

<xarray.Dataset>
Dimensions:    (time: 8766, bnds: 2, lon: 144, lat: 73, level: 1)
Coordinates:
  * time       (time) datetime64[ns] 1998-01-01 1998-01-02 ... 2021-12-31
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * level      (level) float32 850.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(365, 2), meta=np.ndarray>
    uwnd       (time, level, lat, lon) float32 dask.array<chunksize=(365, 1, 73, 144), meta=np.ndarray>
Attributes:
    CDI:            Climate Data Interface version 1.9.10 (https://mpimet.mpg...
    Conventions:    CF-1.0
    source:         NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Model
    institution:    National Centers for Environmental Prediction
    title:          Daily NCEP/DOE Reanalysis 2
    history:        Tue Jan 04 11:04:24 2022: cdo select,level=850 uwnd.1998....
    comments:       Data is from \nNCEP/DOE AMIP-II Reanalysis (Reanalysis-2)...
    platform:       Model
    dataset_title:  NCEP-DOE AMIP-II Reanalysis
    References:     https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.rean...
    source_url:     http://www.cpc.ncep.noaa.gov/products/wesley/reanalysis2/
    CDO:            Climate Data Operators version 1.9.10 (https://mpimet.mpg...

我們發現，uwnd已經是float32的格式了，也就是說，xarray會自動轉譯short成為float，我們不需要再做額外處理。

另外一種方法是

In [5]:
uds = xr.open_mfdataset( 'data/ncep_r2_uv850/u850.*.nc',                                       
                          combine = "nested",               
                          concat_dim='time',                               
                          parallel=True                
                         ) 
uds

<xarray.Dataset>
Dimensions:    (time: 8766, bnds: 2, lon: 144, lat: 73, level: 1)
Coordinates:
  * time       (time) datetime64[ns] 1998-01-01 1998-01-02 ... 2021-12-31
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * level      (level) float32 850.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(365, 2), meta=np.ndarray>
    uwnd       (time, level, lat, lon) float32 dask.array<chunksize=(365, 1, 73, 144), meta=np.ndarray>
Attributes:
    CDI:            Climate Data Interface version 1.9.10 (https://mpimet.mpg...
    Conventions:    CF-1.0
    source:         NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Model
    institution:    National Centers for Environmental Prediction
    title:          Daily NCEP/DOE Reanalysis 2
    history:        Tue Jan 04 11:04:24 2022: cdo select,level=850 uwnd.1998....
    comments:       Data is from \nNCEP/DOE AMIP-II Reanalysis (Reanalysis-2)...
    platform:       Model
    dataset_title:  NCEP-DOE AMIP-II Reanalysis
    References:     https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.rean...
    source_url:     http://www.cpc.ncep.noaa.gov/products/wesley/reanalysis2/
    CDO:            Climate Data Operators version 1.9.10 (https://mpimet.mpg...

兩者結果一模一樣。那有什麼差別呢？在第一種方法中，`combine`引數使用的是`by_coords`，也就是xarray會嘗試著自動排列檔案合併的順序；第二種方法使用的是`combine = nested`，在這種方法中需要指定合併的順序，因此需要多一個引數`concat_dim='time'`，也就是指定xarray在哪個坐標軸上合併。詳細說明可參考[xarray網站 - Combining data](https://xarray.pydata.org/en/stable/user-guide/combining.html#combining-along-multiple-dimensions)。

- `combine_nested()`: requires specifying the order in which the objects should be combined. E.g.: a linearly-increasing ‘time’ dimension coordinate.

- `combine_by_coords()`: attempts to infer this ordering automatically from the coordinates in the data.

## 開啟GRIB檔案

GRIB (GRIdded Binary)格式是以二進位來儲存資料的檔案格式，且也記載了網格資訊如時間、經緯度、垂直層等，xarray也支援開啟這類檔案 (需要安裝`cfgrib`套件)，使用方式如下：

```
ds_grib = xr.open_dataset("example.grib", engine="cfgrib")
```